In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 4.5 MB/s 
     |████████████████████████████████| 511.7 MB 6.2 kB/s 
     |████████████████████████████████| 1.6 MB 56.8 MB/s 
     |████████████████████████████████| 5.8 MB 50.5 MB/s 
     |████████████████████████████████| 438 kB 71.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [ ]:
import pandas as pd

df = pd.read_csv('/content/combined_newcsv.csv', skipinitialspace=True)
df = df.drop(['Unnamed: 0'], axis=1)
df

,Review Rate,Review Text
0,5 stars,Was here for lunch. Had ordered a pork BBQ set...
1,5 stars,I will give 10 stars if permitted! Food is gre...
2,5 stars,I just keep returning. ❤️🌟This is absolutely m...
3,5 stars,"Excellent service, reasonably priced set meal ..."
4,5 stars,Throwback to one of the better Korean BBQs I h...
...,...,...
177514,5 stars,(Translated by Google) The sea breeze blows co...
177515,4 stars,"(Translated by Google) Not bad, very comfortab..."
177516,5 stars,One of the earliest specialize in White Bee Ho...
177517,5 stars,White beehoon is the signature. You don't eat ...


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

'''
num_classes = len(df["Review Rate"].value_counts())

colors = plt.cm.Dark2(np.linspace(0, 1, num_classes))
iter_color = iter(colors)

df['Review Rate'].value_counts().plot.barh(title="Reviews for each rating (n, %)", 
                                                 ylabel="Ratings",
                                                 color=colors,
                                                 figsize=(9,9))

for i, v in enumerate(df['Review Rate'].value_counts()):
  c = next(iter_color)
  plt.text(v, i,
           " "+str(v)+", "+str(round(v*100/df.shape[0],2))+"%", 
           color=c, 
           va='center', 
           fontweight='bold')
  '''

'\nnum_classes = len(df["Review Rate"].value_counts())\n\ncolors = plt.cm.Dark2(np.linspace(0, 1, num_classes))\niter_color = iter(colors)\n\ndf[\'Review Rate\'].value_counts().plot.barh(title="Reviews for each rating (n, %)", \n                                                 ylabel="Ratings",\n                                                 color=colors,\n                                                 figsize=(9,9))\n\nfor i, v in enumerate(df[\'Review Rate\'].value_counts()):\n  c = next(iter_color)\n  plt.text(v, i,\n           " "+str(v)+", "+str(round(v*100/df.shape[0],2))+"%", \n           color=c, \n           va=\'center\', \n           fontweight=\'bold\')\n  '

In [ ]:
df['Labels'] = df['Review Rate'].str.replace('star', '')
df['Labels'] = df['Labels'].str.replace('s', '')
df['Labels'] = df['Labels'].str.strip()
df['Labels'] = df['Labels'].astype(int)
df['Labels'] = df['Labels'].map(lambda x: x-1)
df = df.drop(["Review Rate"], axis=1)
df['Review Text'] = df["Review Text"].str.split('\(Original\)').str[-1]
df['Review Text'] = df["Review Text"].str.replace('\n', ' ')
df['Review Text'] = df["Review Text"].str.replace('\(Translated by Google\)', '')
df['Review Text'] = df["Review Text"].str.strip()
df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


,Review Text,Labels
0,Was here for lunch. Had ordered a pork BBQ set...,4
1,I will give 10 stars if permitted! Food is gre...,4
2,I just keep returning. ❤️🌟This is absolutely m...,4
3,"Excellent service, reasonably priced set meal ...",4
4,Throwback to one of the better Korean BBQs I h...,4
...,...,...
177514,"The sea breeze blows cool, the park pavilion c...",4
177515,"Not bad, very comfortable environment, the foo...",3
177516,One of the earliest specialize in White Bee Ho...,4
177517,White beehoon is the signature. You don't eat ...,4


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")


def get_embeddings(sentences):
  '''return BERT-like embeddings of input text
  Args:
    - sentences: list of strings
  Output:
    - BERT-like embeddings: tf.Tensor of shape=(len(sentences), 768)
  '''
  preprocessed_text = preprocessor(sentences)
  return encoder(preprocessed_text)['pooled_output']

In [ ]:
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


def plot_similarity(features, labels):
  """Plot a similarity matrix of the embeddings."""
  cos_sim = cosine_similarity(features)
  fig = plt.figure(figsize=(10,8))
  sns.set(font_scale=1.2)
  cbar_kws=dict(use_gridspec=False, location="left")
  g = sns.heatmap(
      cos_sim, xticklabels=labels, yticklabels=labels,
      vmin=0, vmax=1, annot=True, cmap="Blues", 
      cbar_kws=cbar_kws)
  g.tick_params(labelright=True, labelleft=False)
  g.set_yticklabels(labels, rotation=0)
  g.set_title("Semantic Textual Similarity")

In [ ]:
reviews = ["pcs queue outlet potato takeaway sauce promotion oil cashier drink",# KFC
           "fried chicken outlet order takeaway tables biscuits drinks app scones cheese fries",
           "lor mee affordable bee hoon value yam mee sua beancurd skin fried fish clams", # Heng Hua Restaurant
           "delivery orders building mashed potatoes", #Texas chicken
           "ramen restaurant price side dishes broth salty tom yum dinner pork kids meal condiments oily ", #Ajisen
           "friendly affordable waitress aunty vaccination bubble tea crew salted egg enthusiasm soft shell crab", #Takagi
           "ramen restaurant price drink table experience outlet value chicken salty attitude money ", #Ajisen
           "western restaurant chicken western food medium rare sides queue outlet value chop attitude coleslaw", #Astons
           "western restaurant value baked potato fish and chips medium rare ribeye nostalgic hot plate lobster bisque sour cream spring onion", #Jacks Place
           "pizza delivery",
           "fast food",
           "coffee shop",
           "hawker centre"
            ]

embed = get_embeddings(reviews)
cosine_similarity(
    [embed[4]],
    embed[:]
)

array([[0.7480728 , 0.7634502 , 0.80835617, 0.7231117 , 0.99999976,
        0.8164682 , 0.8799449 , 0.80154276, 0.8252213 , 0.6471548 ,
        0.671119  , 0.65455854, 0.62644583]], dtype=float32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPool1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda
from tensorflow.keras.layers import Dense, Input, Flatten,SpatialDropout1D

In [ ]:
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [ ]:
import math
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [ ]:
n_epochs = 5

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      balanced_recall,
      balanced_precision,
      balanced_f1_score
]

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 2,
                                                      restore_best_weights = True)

optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.001,
    clipnorm=1.0)

model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = METRICS)

model_fit = model.fit(x_train, 
                      y_train, 
                      epochs = n_epochs,
                      validation_data = (x_val, y_val),
                      callbacks = [earlystop_callback])

Epoch 1/5
4882/4882 [==============================] - 1744s 354ms/step - loss: 0.9125 - accuracy: 0.6095 - balanced_recall: 0.2982 - balanced_precision: 0.3854 - balanced_f1_score: 0.3312 - val_loss: 0.8830 - val_accuracy: 0.6243 - val_balanced_recall: 0.3318 - val_balanced_precision: 0.4362 - val_balanced_f1_score: 0.3716
Epoch 2/5
4882/4882 [==============================] - 1719s 352ms/step - loss: 0.8841 - accuracy: 0.6188 - balanced_recall: 0.3255 - balanced_precision: 0.4120 - balanced_f1_score: 0.3587 - val_loss: 0.8900 - val_accuracy: 0.6186 - val_balanced_recall: 0.3399 - val_balanced_precision: 0.3944 - val_balanced_f1_score: 0.3618
Epoch 3/5
4882/4882 [==============================] - 1721s 352ms/step - loss: 0.8821 - accuracy: 0.6195 - balanced_recall: 0.3308 - balanced_precision: 0.4175 - balanced_f1_score: 0.3642 - val_loss: 0.8645 - val_accuracy: 0.6289 - val_balanced_recall: 0.3174 - val_balanced_precision: 0.4144 - val_balanced_f1_score: 0.3544
Epoch 4/5
4882/4882 [=

In [ ]:
model.save("/models/text_classifier_v1")

INFO:tensorflow:Assets written to: /models/text_classifier_v1/assets


INFO:tensorflow:Assets written to: /models/text_classifier_v1/assets


In [ ]:
!zip -r /content/model.zip /models/text_classifier_v1

  adding: models/text_classifier_v1/ (stored 0%)
  adding: models/text_classifier_v1/keras_metadata.pb (deflated 84%)
  adding: models/text_classifier_v1/variables/ (stored 0%)
  adding: models/text_classifier_v1/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: models/text_classifier_v1/variables/variables.index (deflated 80%)
  adding: models/text_classifier_v1/assets/ (stored 0%)
  adding: models/text_classifier_v1/assets/bert_vocab_500k.external.111lang.txt (deflated 43%)
  adding: models/text_classifier_v1/assets/cased_vocab.txt (deflated 43%)
  adding: models/text_classifier_v1/saved_model.pb (deflated 91%)


In [ ]:
from google.colab import files
files.download('/content/model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow import keras
new_model = keras.models.load_model("/models/text_classifier_v1", compile=False)

# test predictions
[np.argmax(pred) for pred in new_model.predict(reviews)]
# output: [3, 1, 0]

1/1 [==============================] - 1s 1s/step


[4, 4, 0]

# New section